In [8]:
import csv
import codecs
from tqdm import tqdm
from urllib.request import urlretrieve
import json

In [9]:
lines = None
with open('../data/GO/go.obo') as f:
    lines = f.readlines()

In [10]:
cnt = 0
go_dic = {}
for line in lines:
    # if cnt == 2:
    #     break
    line.strip()
    if line.find('id:') == 0:
        go = line[line.find('GO:'):].strip()
        # print(go)
        # cnt += 1
    elif line.find('namespace:') == 0:
        namespace = line[10:].strip()
        # print(namespace)
        # cnt += 1
        go_dic[go] = namespace

    

In [11]:
cnt = 0
go_name_dic = {}
for line in lines:
    # if cnt == 2:
    #     break
    line.strip()
    if line.find('id:') == 0:
        go = line[line.find('GO:'):].strip()
        # print(go)
        # cnt += 1
    elif line.find('name:') == 0:
        name = line[5:].strip()
        # print(namespace)
        # cnt += 1
        go_name_dic[go] = name


In [12]:
len(go_name_dic)

47596

In [13]:
go_dic['GO:0050026']

'molecular_function'

In [14]:
go_rows = []
with codecs.open('../data/GO/pdb_chain_go.csv', encoding='utf-8') as f:
    for go_row in csv.DictReader(f, skipinitialspace=True):
        go_rows.append(go_row)
print(len(go_rows))

8650440


In [15]:
go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'SP_PRIMARY': 'P02185',
 'WITH_STRING': 'UniProtKB-KW:KW-0479',
 'EVIDENCE': 'IEA',
 'GO_ID': 'GO:0046872'}

In [16]:
pdb_chain_go_lines = None
with open('../data/GO/pdb_chain_go.csv', encoding='utf-8') as f:
    pdb_chain_go_lines = f.readlines()
cnt = 0
pdb_chain_go_rows = []
for line in tqdm(pdb_chain_go_lines):
    cnt += 1
    if cnt == 1:
        continue
    elements = line.split(',')
    row = {}
    row['PDB'] = elements[0]
    row['CHAIN'] = elements[1]
    row['uniprot'] = elements[2]
    row['GO_ID'] = elements[-1].strip()
    row['EVIDENCE'] = elements[-2]
    pdb_chain_go_rows.append(row)
print(cnt)

100%|██████████| 8650441/8650441 [00:38<00:00, 225119.03it/s]

8650441


In [17]:
pdb_chain_go_rows[1]

{'PDB': '101m',
 'CHAIN': 'A',
 'uniprot': 'P02185',
 'GO_ID': 'GO:0005344',
 'EVIDENCE': 'IEA'}

In [18]:
go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'SP_PRIMARY': 'P02185',
 'WITH_STRING': 'UniProtKB-KW:KW-0479',
 'EVIDENCE': 'IEA',
 'GO_ID': 'GO:0046872'}

In [24]:
origin_go_id = []
original_datas = {}
with open('../data/GeneOntology/nrPDB-GO_annot.tsv') as f:
    go_origin_lines = f.readlines()
for line in go_origin_lines[12:]:
    data = line.split('\t')
    original_datas[data[0]] = {}
    original_datas[data[0]]['molecular_function'] = data[1].split(',')
    original_datas[data[0]]['biological_process'] = data[2].split(',')
    original_datas[data[0]]['cellular_component'] = data[3].split(',')
for i in [1,5,9]:
    origin_go_id.extend(go_origin_lines[i].split('\t'))
origin_go_id = [i.strip() for i in origin_go_id]

In [20]:
with open('../output_info/gene_ontology_uniprots.json') as f:
    original_uniprots = json.load(f)
need_uniprots = []
for key in original_datas.keys():
    try:
        original_datas[key]['uniprot'] = original_datas[key]
    except:
        need_uniprots.append(key)

In [25]:
from tqdm import tqdm
# go_annot = {}
for row in tqdm(pdb_chain_go_rows):
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    # pdb_chain = pdb_chain.upper()
    # print(row['GO_ID'])
    try:
        if pdb_chain in original_datas.keys():
            if row['GO_ID'] in origin_go_id:
                original_datas[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
        else:
            original_datas[pdb_chain] = {}
            original_datas[pdb_chain]['uniprot'] = row['uniprot']
            original_datas[pdb_chain]['molecular_function'] = []
            original_datas[pdb_chain]['biological_process'] = []
            original_datas[pdb_chain]['cellular_component'] = []
            if row['GO_ID'] in origin_go_id:
                original_datas[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
    except Exception as e:
        print(e, pdb_chain, row['GO_ID'])

100%|██████████| 8650440/8650440 [16:14<00:00, 8874.98it/s]  


In [22]:
from tqdm import tqdm
go_annot = {}
for row in tqdm(pdb_chain_go_rows):
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    # pdb_chain = pdb_chain.upper()
    # print(row['GO_ID'])
    try:
        if pdb_chain in go_annot.keys():
            if row['GO_ID'] in origin_go_id:
                go_annot[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
        else:
            go_annot[pdb_chain] = {}
            go_annot[pdb_chain]['uniprot'] = row['uniprot']
            go_annot[pdb_chain]['molecular_function'] = []
            go_annot[pdb_chain]['biological_process'] = []
            go_annot[pdb_chain]['cellular_component'] = []
            if row['GO_ID'] in origin_go_id:
                go_annot[pdb_chain][go_dic[row['GO_ID']]].append(row['GO_ID'])
    except Exception as e:
        print(e, pdb_chain, row['GO_ID'])

  1%|▏         | 115351/8650440 [00:07<09:36, 14810.26it/s]


KeyboardInterrupt: 

In [27]:
# print(go_annot['1SZ7-A'])
print(len(original_datas))

443965


In [28]:
with open('../data/GO/all_download.json') as f:
    download_pdb_go = json.load(f)

In [ ]:
import os
original_download_pdb = os.listdir('../data/GeneOntology/all')

In [ ]:
download_pdb_go = [pdbid.upper() for pdbid in download_pdb_go]
download_pdb_go = list(set(download_pdb_go))

In [ ]:
import csv
go_tsv = []
for key, value in tqdm(go_annot.items()):
    if key[:4] in download_pdb_go:
        if len(value['molecular_function']) == 0 and len(value['biological_process']) == 0 and len(value['cellular_component']) == 0:
            continue
        go_tsv.append([key, ','.join(list(set(value['molecular_function']))), ','.join(list(set(value['biological_process']))), ','.join(list(set(value['cellular_component'])))])

100%|██████████| 428977/428977 [21:14<00:00, 336.55it/s] 


In [ ]:
len(go_tsv)

284796

In [ ]:
mf  = []
bp = []
cc = []
for key, value in tqdm(go_annot.items()):
    if key[:4] in download_pdb_go:
        mf.extend(value['molecular_function'])
        bp.extend(value['biological_process'])
        cc.extend(value['cellular_component'])
mf = list(set(mf))
bp = list(set(bp))
cc = list(set(cc))
print(len(mf), len(bp), len(cc))

100%|██████████| 428977/428977 [21:18<00:00, 335.42it/s]


426 1363 255


In [ ]:
print(len(go_tsv))

284796


In [ ]:
mf_name = [go_name_dic[go_mf] for go_mf in mf]
bp_name = [go_name_dic[go_bp] for go_bp in bp]
cc_name = [go_name_dic[go_cc] for go_cc in cc]

In [ ]:
type_list = ['molecular_function', 'biological_process', 'cellular_component']
go_type_list = {
    'molecular_function': [mf, mf_name],
    'biological_process': [bp, bp_name],
    'cellular_component': [cc, cc_name]
}
annot = ""
for t in type_list:
    annot += '### GO-terms ({})'.format(t) +'\n'
    annot += ('\t'.join(go_type_list[t][0]) + '\n')
    annot += '### GO-names ({})'.format(t) +'\n'
    annot += ('\t'.join(go_type_list[t][1]) + '\n')

In [ ]:
with open('../data/GO/nrPDB-GO_annot.tsv', 'w') as f:
    f.writelines(annot)
    for row in go_tsv:
        f.writelines('\t'.join(row) + '\n')

In [ ]:
for row in go_tsv:
    if '7NS4' in row[0]:
        print(row)

['7NS4-b', 'GO:0004842,GO:0061630,GO:0008270', 'GO:0006511,GO:0016567,GO:0043161', 'GO:0005634,GO:0005777,GO:0005829']
['7NS4-i', 'GO:0004842,GO:0061630', 'GO:0016567,GO:0043066,GO:0043161', 'GO:0005634']


In [5]:
go_uniprot_dic = {}
for row in tqdm(pdb_chain_go_rows):
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain not in go_uniprot_dic.keys():
        go_uniprot_dic[pdb_chain] = []
    go_uniprot_dic[pdb_chain].append(row['uniprot'])
for k, v in tqdm(go_uniprot_dic.items()):
    go_uniprot_dic[k] = list(set(v))

100%|██████████| 428977/428977 [00:01<00:00, 242920.47it/s]


In [6]:
for key, value in go_uniprot_dic.items():
    if len(value) > 1:
        print(key, value)

1F11-A ['P01837', 'P01665']
1F11-C ['P01837', 'P01665']
1HSJ-A ['P0AEX9', 'Q9F0R1']
1HSJ-B ['P0AEX9', 'Q9F0R1']
1PUO-A ['P30440', 'P30438']
1PUO-B ['P30440', 'P30438']
1ZVO-A ['P0CG04', 'P01700']
1ZVO-B ['P0CG04', 'P01700']
2JAD-A ['P25373', 'P42212']
3AGP-A ['P0A6P3', 'P0A6N3', 'P14647']
3AGQ-A ['P0A6P3', 'P0A6N3', 'P14647']
3AL0-C ['Q9X2I8', 'Q9WY94']
3AVT-A ['P0A6P3', 'P0A6N3', 'P14647']
3AVU-A ['P0A6P3', 'P0A6N3', 'P14647']
3AVV-A ['P0A6P3', 'P0A6N3', 'P14647']
3AVW-A ['P0A6P3', 'P0A6N3', 'P14647']
3AVX-A ['P0A6P3', 'P0A6N3', 'P14647']
3AVY-A ['P0A6P3', 'P0A6N3', 'P14647']
3DM0-A ['P0AEX9', 'O24456']
3F83-A ['P0CK93', 'P25734']
3J9P-A ['P0AEX9', 'O75762']
3J9P-B ['P0AEX9', 'O75762']
3J9P-C ['P0AEX9', 'O75762']
3J9P-D ['P0AEX9', 'O75762']
3MMP-A ['P0CE48', 'P0A6P1']
3MMP-C ['P0CE48', 'P0A6P1']
3PY7-A ['P0AEX9', 'P06931']
3TSY-A ['Q2HY10', 'Q42524']
3U0W-L ['P01834', 'P01647']
3VNU-A ['P0A6P3', 'P0A6N3', 'P14647']
3VNV-A ['P0A6P3', 'P0A6N3', 'P14647']
4BL8-A ['P0AEX9', 'Q9UMX1']
4BL8

In [ ]:
go_uniprots = {}
for go in go_tsv:
    pdbid = go[0]
    go_uniprots[pdbid] = go_uniprot_dic[pdbid]

In [ ]:
with open('../output_info/go_uniprots.json', 'w') as f:
    json.dump(go_uniprots, f)

In [ ]:
pdb_chain_go_rows[0]

{'PDB': '101m',
 'CHAIN': 'A',
 'uniprot': 'P02185',
 'GO_ID': 'GO:0046872',
 'EVIDENCE': 'IEA'}

In [4]:
pdb_chain_uniprots = {}
for row in pdb_chain_go_rows:
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain in pdb_chain_uniprots.keys():
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['uniprot']
    else:
        pdb_chain_uniprots[pdb_chain] = {}
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['uniprot']

In [5]:
ec_rows = []
with codecs.open('../data/EC/pdb_chain_enzyme.csv', encoding='utf-8') as f:
    for row in csv.DictReader(f, skipinitialspace=True):
        ec_rows.append(row)
print(len(ec_rows))

323404


In [6]:
ec_rows[0]

{'PDB': '102l', 'CHAIN': 'A', 'ACCESSION': 'P00720', 'EC_NUMBER': '3.2.1.17'}

In [7]:
for row in tqdm(ec_rows):
    if row['PDB'] == '1ir1':
        print(row)
    pdb_chain = row['PDB'].upper() + '-' + row['CHAIN']
    if pdb_chain in pdb_chain_uniprots.keys():
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['ACCESSION']
    else:
        pdb_chain_uniprots[pdb_chain] = {}
        pdb_chain_uniprots[pdb_chain][row['CHAIN']] = row['ACCESSION']

  7%|▋         | 22555/323404 [00:00<00:01, 225524.41it/s]

{'PDB': '1ir1', 'CHAIN': 'A', 'ACCESSION': 'P00875', 'EC_NUMBER': '4.1.1.39'}
{'PDB': '1ir1', 'CHAIN': 'B', 'ACCESSION': 'P00875', 'EC_NUMBER': '4.1.1.39'}
{'PDB': '1ir1', 'CHAIN': 'C', 'ACCESSION': 'P00875', 'EC_NUMBER': '4.1.1.39'}
{'PDB': '1ir1', 'CHAIN': 'D', 'ACCESSION': 'P00875', 'EC_NUMBER': '4.1.1.39'}


100%|██████████| 323404/323404 [00:00<00:00, 333782.25it/s]


In [8]:
import json
with open('../data/ProtFunc/uniformed_labels.json') as f:
    protfun_dic = json.load(f)

In [11]:
for key, value in tqdm(protfun_dic.items()):
    pdb_chain = key
    chain_id = key.split('-')[-1]
    try:
        if pdb_chain in pdb_chain_uniprots.keys():
            pdb_chain_uniprots[pdb_chain][chain_id] = value["uniprots"][0]
        else:
            pdb_chain_uniprots[pdb_chain] = {}
            pdb_chain_uniprots[pdb_chain][chain_id] = value["uniprots"][0]
    except:
        print(key,value)
        

 95%|█████████▌| 35695/37400 [00:00<00:00, 179570.62it/s]

5JN1-A {'uniprots': [], ',ec': [-1], 'reaction': [[178]], 'go': [-1], 'ppi': -1, 'lba': -1}
5IOL-D {'uniprots': [], ',ec': [-1], 'reaction': [[83]], 'go': [-1], 'ppi': -1, 'lba': -1}
5IOL-A {'uniprots': [], ',ec': [-1], 'reaction': [[83]], 'go': [-1], 'ppi': -1, 'lba': -1}
5IOL-C {'uniprots': [], ',ec': [-1], 'reaction': [[83]], 'go': [-1], 'ppi': -1, 'lba': -1}
5IOL-K {'uniprots': [], ',ec': [-1], 'reaction': [[83]], 'go': [-1], 'ppi': -1, 'lba': -1}
5IOM-A {'uniprots': [], ',ec': [-1], 'reaction': [[83]], 'go': [-1], 'ppi': -1, 'lba': -1}
4IQQ-A {'uniprots': [], ',ec': [-1], 'reaction': [[238]], 'go': [-1], 'ppi': -1, 'lba': -1}
4IQQ-C {'uniprots': [], ',ec': [-1], 'reaction': [[238]], 'go': [-1], 'ppi': -1, 'lba': -1}
5URR-D {'uniprots': [], ',ec': [-1], 'reaction': [[235]], 'go': [-1], 'ppi': -1, 'lba': -1}
5BYH-B {'uniprots': [], ',ec': [-1], 'reaction': [[48]], 'go': [-1], 'ppi': -1, 'lba': -1}
4WTO-A {'uniprots': [], ',ec': [-1], 'reaction': [[251]], 'go': [-1], 'ppi': -1, 'lba'

100%|██████████| 37400/37400 [00:00<00:00, 169213.10it/s]


In [12]:
pdb_chain_uniprots['1IR1-V']

{'V': 'Q43832'}

In [13]:
with open('../output_info/uniprot_dict_all_origin.json') as f:
    uniprots_all = json.load(f)

In [14]:
with open('../datasets/MultiTaskNew/test.txt') as f:
    test_list  = f.readlines()
with open('../datasets/MultiTaskNew/val.txt') as f:
    val_list  = f.readlines()
with open('../datasets/MultiTaskNew/train.txt') as f:
    train_list  = f.readlines()
with open('../datasets/MultiTaskNew/train_all.txt') as f:
    train_all_list  = f.readlines()

In [15]:
all_list = test_list + val_list + train_list + train_all_list
for i in all_list:
    i = i[:-1]
    if i not in uniprots_all.keys():
        uniprots_all[i] = pdb_chain_uniprots[i]

In [16]:
with open('../output_info/uniprot_dict_all_reaction.json', 'w') as f:
    json.dump(uniprots_all, f)

In [ ]:
test_go = '''155C-A	GO:0009055,GO:0046906,GO:0020037	GO:0006091,GO:0022900	
'''

In [ ]:
# test_go = test_go.split('\t')
test_go

In [ ]:
test_go[3].find('48208')

In [ ]:
nr_lines = None
with open('../data/nrPDB-GO_annot.tsv') as f:
    nr_lines = f.readlines()

In [ ]:
go_term_dic = {'molecular_function': [], 'biological_process': [], 'cellular_component': []}
flag = 0
for idx, line in enumerate(nr_lines):
    if line.find('### GO-terms') == 0:
        gos = nr_lines[idx + 1].split('\t')
        gos = [go_id.strip() for go_id in gos]
        if flag == 0:
            go_term_dic['molecular_function'] = gos
        elif flag == 1:
            go_term_dic['biological_process'] = gos
        elif flag == 2:
            go_term_dic['cellular_component'] = gos
        flag += 1

In [ ]:
go_terms = {'molecular_function': [], 'biological_process': [], 'cellular_component': []}
for i in range(13, len(nr_lines)):
    GO_TERM = nr_lines[i][:-1].split('\t')
    if GO_TERM[1]!= '':
        go_terms['molecular_function'].extend(GO_TERM[1].split(','))
    if GO_TERM[2]!= '':
        go_terms['biological_process'].extend(GO_TERM[2].split(','))
    if GO_TERM[3]!= '':
        go_terms['cellular_component'].extend(GO_TERM[3].split(','))

In [ ]:
go_terms['molecular_function'] = list(set(go_terms['molecular_function']))
go_terms['biological_process'] = list(set(go_terms['biological_process']))
go_terms['cellular_component'] = list(set(go_terms['cellular_component']))

In [ ]:
if go_term_dic['cellular_component'].sort() == go_terms['cellular_component'].sort():
    print('true')